In [1]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torchmetrics import R2Score

dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
torch.set_float32_matmul_precision('medium')
print(dev)

e:\miniconda3\envs\AI\lib\site-packages\torch\distributed\_shard\partial_tensor.py:40: UserWarning: Please use DTensor instead and we are deprecating ShardedTensor.
  warnings.warn(DEPRECATE_MSG)
e:\miniconda3\envs\AI\lib\site-packages\torch\distributed\_shard\replicated_tensor.py:20: UserWarning: Please use DTensor instead and we are deprecating ShardedTensor.
  warnings.warn(DEPRECATE_MSG)


cuda


In [2]:
class CNNModel(nn.Module):
      def __init__(self, in_chanels=3, out_chanels=1):
            super(CNNModel, self).__init__()
            self.conv_layer1 = self._conv_layer_set(in_chanels, 16)
            self.conv_layer2 = self._conv_layer_set(16, 32)
            self.conv_layer3 = self._conv_layer_set(32, 64)
            self.conv_layer4 = self._conv_layer_set(64, 128)
            self.linear1 = self._dense_layer_set(128*8*8*8, 128)
            self.linear2 = self._dense_layer_set(128, 64)
            self.linear3 = self._dense_layer_set(64, out_chanels)
            
            
      def _conv_layer_set(self, in_c, out_c):
            conv_layer = nn.Sequential(
            nn.Conv3d(in_c, out_c, kernel_size=3,padding="same"),
            nn.LeakyReLU(),
            nn.MaxPool3d((2, 2, 2)),
            )
            return conv_layer
      def _dense_layer_set(self, in_c, out_c,drop= 0.2):
            dense_layer = nn.Sequential(
            nn.Linear(in_c, out_c),
            nn.LeakyReLU(),
            nn.BatchNorm1d(out_c),
            nn.Dropout(p=drop)            
            )
            return dense_layer

      def forward(self, x):
            x = self.conv_layer1(x)
            x = self.conv_layer2(x)
            x = self.conv_layer3(x)
            x = self.conv_layer4(x)
            # print(x.shape)
            x = x.view(x.size(0), -1)            
            x = self.linear1(x)
            x = self.linear2(x)
            x = self.linear3(x)
            return x
            
input_size = 128

x = torch.randn(10, 3, input_size, input_size, input_size)

model = CNNModel()
y = model(x)
y.shape


      

torch.Size([10, 1])

In [10]:
# import pytorch dataset
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np


class LoadMolecules(Dataset):
    def __init__(self, molecules_root, band_root, max_samples=100):
        self.root = molecules_root
        self.files = list(os.listdir(molecules_root))
        if max_samples:
            self.files = self.files[:min(max_samples, len(self.files))]
        self.bandgap = np.loadtxt(
            f'{band_root}/bandgaps.csv', dtype=np.float32)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        data = np.load(f'{self.root}/{self.files[idx]}').astype(np.float32)
        data = np.transpose(data, (3, 0, 1, 2))

        return data, self.bandgap[idx]


class InMemoryDataModule(pl.LightningDataModule):
    def __init__(self, data, batch_size):
        super().__init__()
        self.data = data
        self.batch_size = batch_size
        self.train_size =int( len(data)*0.7)
        self.val_size = int(len(data)*.2)
        self.test_size = len(data) - self.train_size - self.val_size
        self.train_data, self.val_data, self.test_data = None, None, None

    def setup(self, stage=None):
        self.train_data, self.val_data, self.test_data = torch.utils.data.random_split(
            self.data, [self.train_size, self.val_size,   self.test_size])

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=False)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size, shuffle=False)


In [4]:
class BandGap(pl.LightningModule):
    def __init__(self, lr=1e-3, save_every_n_epoch=10):
        super(BandGap,self).__init__()
        self.lr = lr
        self.save_hyperparameters()
        self.save_every_n_epoch = save_every_n_epoch
        self.loss = nn.MSELoss()
        self.r2score = R2Score()
        self.model = CNNModel().to(dev)
    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.unsqueeze(1)
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        r2 = self.r2score(y_hat, y)
        self.log("train_loss", loss, prog_bar=True, on_step=True)
        self.log("train_r2", r2, prog_bar=True, on_step=True)
        return {"loss": loss, "log": {"train_loss": loss, "train_r2": r2}}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.unsqueeze(1)
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        r2 = self.r2score(y_hat, y)
        self.log("val_loss", loss, prog_bar=True, on_step=True)
        self.log("val_r2", r2, prog_bar=True, on_step=True)
        return {"loss": loss, "log": {"val_loss": loss, "val_r2": r2}}

    def test_step(self, batch, batch_idx):
        x, y = batch
        y = y.unsqueeze(1)
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        r2 = self.r2score(y_hat, y)
        self.log("test_loss", loss, prog_bar=True, on_step=True)
        self.log("test_r2", r2, prog_bar=True, on_step=True)
        return {"loss": loss, "log": {"test_loss": loss, "test_r2": r2}}
    

   

In [17]:
data  = LoadMolecules(molecules_root = r"E:\Datasets\BandGap\3d-boxels-molecule-for-bandgap-prediction\Data",band_root =r"E:\Datasets\BandGap\3d-boxels-molecule-for-bandgap-prediction" , max_samples=12500)
# data = [x for x in data]
print(len(data))

12500


In [40]:



checkpoint_callback = ModelCheckpoint(
    dirpath='./checkpoints',
    filename='{epoch}-{val_loss:.2f}-{val_r2:.2f}',
    save_top_k=1,
    monitor='val_loss',
    every_n_epochs =1,
)
# EARLY STOPING callback
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.01,
    patience=10,
    verbose=False,
    mode='min'
)
model = BandGap()
# load checkpoint
model = BandGap.load_from_checkpoint(checkpoint_path=r"checkpoints\1500 moleculas epoch=71-val_loss=0.92.ckpt")


trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=100,log_every_n_steps=1,callbacks=[checkpoint_callback])


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [41]:

torch.cuda.empty_cache()

In [42]:
data_module = InMemoryDataModule(data,32)
trainer.fit(model,data_module)

e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:613: UserWarning: Checkpoint directory E:\Github\band-gap\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type     | Params
-------------------------------------
0 | loss    | MSELoss  | 0     
1 | r2score | R2Score  | 0     
2 | model   | CNNModel | 8.7 M 
-------------------------------------
8.7 M     Trainable params
0         Non-trainable params
8.7 M     Total params
34.757    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
trainer.validate(model, data_module)
trainer.test(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     val_loss_epoch         1.0289413928985596
      val_r2_epoch          -0.1823505014181137
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_loss_epoch        1.0611859560012817
      test_r2_epoch         -0.1461295485496521
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_epoch': 1.0611859560012817, 'test_r2_epoch': -0.1461295485496521}]

In [39]:
x , y = data_module.test_data[742]
# x to tensor
x = torch.tensor(x).unsqueeze(0).to(dev)
# model to eval
model.eval()
out = model(x)
print(out, y)

tensor([[3.3103]], device='cuda:0', grad_fn=<NativeBatchNormBackward0>) 2.90567
